# Example of BOMD MD (NVE, NVT)

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin

torch.set_default_dtype(torch.float64)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                           # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                           # [1], adaptive mixing
                                           # [1, K, L, M] # advanced adaptive mixing.
                                           # First, it does linear mixing for M steps. Mixing coeff is K the first 5 SCF steps.
                                           # Then it incrementally goes to L and becomes L at M-5 SCF step. From M-5 to M it's equal to L. After M'th SCF step, adaptive mixing begins.
                                           # 
                                           # [2], adaptive mixing, then pulay
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule, Temperature = 400)
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   290.00   1.499427e-01 -1.353970e+00 -1.204028e+00 ||    404.39   2.090863e-01 1.487221e+00 1.696307e+00 || 
     2   255.64   1.321770e-01 -1.336068e+00 -1.203891e+00 ||    328.79   1.699998e-01 1.526568e+00 1.696568e+00 || 
     3   206.99   1.070235e-01 -1.310670e+00 -1.203647e+00 ||    290.27   1.500832e-01 1.546576e+00 1.696659e+00 || 
     4   151.66   7.841607e-02 -1.281754e+00 -1.203338e+00 ||    291.64   1.507887e-01 1.545795e+00 1.696584e+00 || 
     5    99.07   5.122545e-02 -1.254260e+00 -1.203034e+00 ||    330.64   1.709560e-01 1.525398e+00 1.696354e+00 || 
     6    58.41   3.019842e-02 -1.233009e+00 -1.202811e+00 ||    400.37   2.070089e-01 1.488982e+00 1.695991e+00 || 
     7    36.37   1.880430e-02 -1.221529e+00 -1.202725e+00 ||    489.91   2.533011e-01 1.442223e+00 1.695524e+00 || 
     8    35.55   1.838281e-02 -1.221176e+00 -1.202793e+00 ||    585.33   3.026373e-01 1.392358e+00 1.694995e+00 || 
     9   

## *MD_BOMD_BASIC.0.xyz* and *MD_BOMD_BASIC.1.xyz* files with general info for each step are created. Coordinates, velocities, forces and charges are recorded in the following sequence: x y z vx vy vz fx fy fz q.

## *MD_BOMD_BASIC.Info.0.txt* and *MD_BOMD_BASIC.Info.1.txt* contain additional info for each step (orbital energies, dipole).

## Scale velocities to adjust kinetic energy and compenstate the energy shift

In [3]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_E_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule, Temperature = 400)
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, control_energy_shift=True, remove_com=[True, 1], Info_log=True)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   308.02   1.592606e-01 -8.742330e-02 7.183732e-02 ||    461.47   2.385972e-01 2.103726e+00 2.342323e+00 || 
     2   354.51   1.832937e-01 -1.114563e-01 7.183732e-02 ||    618.63   3.198575e-01 2.022466e+00 2.342323e+00 || 
     3   456.14   2.358402e-01 -1.640028e-01 7.183732e-02 ||    838.64   4.336092e-01 1.908714e+00 2.342323e+00 || 
     4   609.77   3.152734e-01 -2.434361e-01 7.183732e-02 ||   1103.74   5.706782e-01 1.771645e+00 2.342323e+00 || 
     5   810.22   4.189146e-01 -3.470773e-01 7.183732e-02 ||   1389.57   7.184639e-01 1.623859e+00 2.342323e+00 || 
     6  1049.91   5.428476e-01 -4.710102e-01 7.183732e-02 ||   1664.83   8.607860e-01 1.481537e+00 2.342323e+00 || 
     7  1318.46   6.816973e-01 -6.098599e-01 7.183732e-02 ||   1891.61   9.780370e-01 1.364286e+00 2.342323e+00 || 
     8  1602.22   8.284136e-01 -7.565762e-01 7.183732e-02 ||   2027.17   1.048129e+00 1.294194e+00 2.342323e+00 || 
     9  1884.05  

## Scale velocities to control temperature

In [4]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_T_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule, Temperature = 400)
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, scale_vel=[1,400], remove_com=[True, 1], Info_log=True)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.00   2.068160e-01 -1.033611e-01 1.034549e-01 ||    400.00   2.068160e-01 1.968366e+00 2.175182e+00 || 
     2   400.00   2.068160e-01 -1.452816e-01 6.153438e-02 ||    400.00   2.068160e-01 1.837371e+00 2.044187e+00 || 
     3   400.00   2.068160e-01 -2.087411e-01 -1.925138e-03 ||    400.00   2.068160e-01 1.711593e+00 1.918409e+00 || 
     4   400.00   2.068160e-01 -2.882311e-01 -8.141512e-02 ||    400.00   2.068160e-01 1.594514e+00 1.801330e+00 || 
     5   400.00   2.068160e-01 -3.783976e-01 -1.715816e-01 ||    400.00   2.068160e-01 1.488490e+00 1.695306e+00 || 
     6   400.00   2.068160e-01 -4.746072e-01 -2.677912e-01 ||    400.00   2.068160e-01 1.395227e+00 1.602043e+00 || 
     7   400.00   2.068160e-01 -5.730984e-01 -3.662824e-01 ||    400.00   2.068160e-01 1.316054e+00 1.522870e+00 || 
     8   400.00   2.068160e-01 -6.709300e-01 -4.641140e-01 ||    400.00   2.068160e-01 1.252081e+00 1.458897e+00 || 
     9   40

## Langevin Thermostat

In [5]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Langevin'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# specify damping factor
md =  Molecular_Dynamics_Langevin(damp = 100.0, seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule, Temperature = 400)
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   679.06   3.511032e-01 -1.348199e+00 -9.970962e-01 ||    227.87   1.178194e-01 1.440759e+00 1.558579e+00 || 
     2   614.49   3.177165e-01 -1.317261e+00 -9.995441e-01 ||    228.04   1.179061e-01 1.430736e+00 1.548642e+00 || 
     3   540.60   2.795107e-01 -1.272970e+00 -9.934597e-01 ||    256.45   1.325950e-01 1.402357e+00 1.534952e+00 || 
     4   449.22   2.322669e-01 -1.222749e+00 -9.904823e-01 ||    308.72   1.596200e-01 1.366804e+00 1.526424e+00 || 
     5   356.52   1.843337e-01 -1.173960e+00 -9.896265e-01 ||    369.00   1.907893e-01 1.323201e+00 1.513991e+00 || 
     6   286.49   1.481283e-01 -1.132060e+00 -9.839313e-01 ||    448.26   2.317658e-01 1.280801e+00 1.512567e+00 || 
     7   238.99   1.235651e-01 -1.101742e+00 -9.781766e-01 ||    506.81   2.620386e-01 1.245192e+00 1.507231e+00 || 
     8   224.26   1.159492e-01 -1.086432e+00 -9.704828e-01 ||    529.11   2.735708e-01 1.223009e+00 1.496580e+00 || 
     9   